In [1]:
import pyvisa
import numpy as np
import pandas as pd
import os
import csv
import time

In [2]:
def get_freq_str(freq_list):
    
    """
    Function that transforms the array fo frequency values into a strin of comma separated entries.
    
    The array is converted into a string and the square brakets are removed form it.
    The resulting string will be the list of frequencies to sweep over,
    organized into comma separated entries. 
    
    
    Parameters
    ----------
    
    freq_list: list
                List of frequencies to sweep over. 
                
    Returns
    -------
    
    freq_str: str
              String of comma separated frequenncies to be inputted into the instrument.
        
    """
    
    freq_str= str(freq_list)
    
    freq_str= freq_str.split('[')[1].split(']')[0]
    
    return freq_str

In [3]:
def get_components(EIS_measurements):
    
    """
    Function to extract the primary and secondary outputs from the instrument. 
    
    The actual output from the instrument is a list of values. For every measurement point collected, 
    four outputs are provided. The first two are the actual measured values and the latter two refer 
    to the status and the bin number. The latter two points are usually zero and will be removed in this funciton. 
    
    
    Parameters
    ----------
    
    EIS_measurements: list
                      List outputted by the buffer memeory from the instrument.
                      
    Returns
    -------
    
    Z_magnitude: list
               List containing the values of the primary output from the instrument.
               In this case it is the magnitude of the measured impedance. 
    Z_phase: list
              List containing the values of the secondary output from the instrument.
              In this case it is the phase of the measured impedance. 
    
    
    Note: For each measurement, 4 entries are generated:

    Entry 1: Impedance Magnitude
    Entry 2: Phase Angle (rad)
    Entry 3&4: 0.0
    
    """
    # remove the empty entries [ 0.0 status and bin no. of every measurement.]
    EIS_measurements= [ item for item in EIS_measurements if item!= 0.0]
    
    # Extract the primary output
    Z_magnitude= [EIS_measurements[i] for i in range(len(EIS_measurements)) if i % 2 == 0]
    
    # Extract the secondary output
    Z_phase= [EIS_measurements[i] for i in range(len(EIS_measurements)) if i % 2 != 0]
    
    return Z_magnitude, Z_phase

In [15]:
def write_file(freq, EIS_measurements, filename, save_location, comment=None):
    
    """
    Function that writes and saves a .csv file containing the results of the measurement. 
    The units of the quantities being saved is indicated as a metadata line.
    Thes are hardcoed in this funciton.
    
    Parameters
    ----------
    
    freq: list
           List of frequencies to sweep over during the experiment.
    EIS_measurements: list 
                       List outputted by the buffer memeory from the instrument.
    filename: str
              The name to use to save the .csv file with.
    save_location: str
                   Path to the folder the file will be save in. The default value is
                   the cwd. 
                   
    Rerurns
    -------
    If the all lines are correctly executed, the function will return a print statement.
                    
    """
    
    
    if not os.path.exists(save_location):
        os.makedirs(save_location)
    
    magnitude, phase= get_components(EIS_measurements)
    to_file= { 'frequency': freq , 'magnitude': magnitude, 'phase' :phase}
    df_to_file= pd.DataFrame.from_dict(to_file)
    
    with open(save_location+filename +'.csv', mode='w',
              newline='') as f:
        f.write('frequency [Hz] \n')
        f.write('magnitude [ohm] \n')
        f.write('phase angle [rad] \n')
        if comment:
            comment= comment
        else:
            comment = 'None'
        f.write('Notes: ' + comment+'\n')
        df_to_file.to_csv(f)
        f.close()
        
    return print('The file was correctly saved')

In [5]:
def freq_gen(high_freq, low_freq, pts_per_decade=10):
    '''
    Function that generates the frequency range used to investigate the
    impedance response of an electrical circuit Frequency Generator with
    logspaced freqencies

    Parameters
    ----------
    high_freq: single value (int or float)
               Initial frequency value (high frequency domain) [Hz]
    high_freq: single value (int or float)
               Final frequency value (low frequency domain) [Hz]
    pts_per_decade: integer
                    Number of frequency decades to be used as range. Default value
              is set to be 10 [-]

    Returns
    ----------
    freq_list: list
               List of frequency [Hz] linearly spaced on a base-ten logarithm scale
    
    '''
    f_decades = int(np.log10(int(high_freq)) - np.log10(low_freq))
    f_range = np.logspace(np.log10(int(high_freq)), np.log10(low_freq),
                          np.around(pts_per_decade*f_decades), endpoint=True)

    freq_list= [ np.round(item,6) for item in f_range]
    return freq_list

In [6]:
def gen_filename(sample, dist, temp):
    
    """
    Function to generate the string to use as the filename of the current measurement
    
    
    Parameters
    ----------
    sample: str
            String indicating the sample under investigation
    dist:  str
           String containing the 
    
    Returns
    -------
    
    filename: str
              String to use as the filename of the current measurement
    
    """
    date = time.strftime('%y%m%d', time.localtime())
    filename = date + '_EIS_' + sample + '_' + dist + '_' + temp
    
    return filename

In [7]:
def EIS_measurement(my_instrument, function, amplitude):

    """
    """
    
    # Clear previous states and resets the instrument settings
    my_instrument.write('*RST; *CLS')

    # Enable the display to update itself when a change is made
    my_instrument.write(':DISP:ENAB')
    time.sleep(2)

    # Configure the instrument to automatically perform continuous
    # measurements
    my_instrument.write(':INIT:CONT')

    # * Switch the instrument triggering function to 'EXT'
    # in order to control the measurements remotely
    my_instrument.write(':TRIG:SOUR EXT')
    time.sleep(2)

    # Choose the function we want to use for the experimental run.
    # In this case we are going to be runnin an Electrochemical Impedance Spectrum by measuring the magnitude and the phase angle - in radians
    my_instrument.write(':FUNC:IMP:TYPE ', function)

    # Set the input voltage amplitude by runnning the following cell.
    my_instrument.write(':VOLT:LEVEL ', amplitude)
    time.sleep(2)

    # Set the measurement time to medium
    my_instrument.write(':APER MED')

    # Set the impedance range to automatic
    my_instrument.write(':FUNC:IMP:RANGE:AUTO ON')
    time.sleep(2)

    # Displays the list page. Once you set the list of
    # freqeuncies you want to scan over, the list page should be automatically updated
    my_instrument.write(':DISP:PAGE LIST')
    my_instrument.write(':FORM:DATA ASC')
    my_instrument.write(':LIST:MODE SEQ')
    time.sleep(2)

    # Set the frequency you want to run the impedance measurement at
    my_instrument.write('LIST:FREQ ', freq_str)
    time.sleep(2)

    # Set the source of the memory to external. 
    my_instrument.write(':MMEM EXT')

    # We are gonna use the memory buffer as I am going to independently
    # save the data into a single file instead of having the instrument
    # do that for me. Giving dimensions to the buffer memory ensures
    # that only the data collected is going to be save. 
    my_instrument.write(':MEM:DIM DBUF, ', str(len(freq_list)))

    # Initialize the memory log for the given frequency list indicated above
    my_instrument.write(':MEM:FILL DBUF')
    time.sleep(2)

    # Triggers the instrument to run the measurement
    my_instrument.write(':TRIG:IMM')
    time.sleep(2)

    # Save the data we just collected into a variable
    EIS_measurements= my_instrument.query_ascii_values(':MEM:READ? DBUF')
    time.sleep(5)

    # * Let's clean the memory buffer and return to the display page
    my_instrument.write(':MEM:CLE DBUF')
    my_instrument.write(':DISP:PAGE MEAS')

    return EIS_measurements

In [8]:
def open_correction(my_instrument):
    
    """
    Function to activating and measuring the open circuit potential
    (OCP) of the current system. The correction will then be active
    on the intrument. Once this funciton is ran, the current OCP
    will be stored in the instrument and unless tuned off,
    the correction will be applied to all the measurements collected after.
    
    
    Parameters
    ----------
    
    my_instrument: pyvisa.resources.usb.USBInstrument
                   Instrument instance obtained throught the Pyvisa package
    
    Returns
    -------
    
    Once the opem measurement is completed, the function will return a 
    print statement confirming the task was completed. 
    """

    # Clear previous states and resets the instrument settings
    my_instrument.write('*RST; *CLS')

    # Enable the display to update itself when a change is made
    my_instrument.write(':DISP:ENAB')
    time.sleep(3)

    # Configure the instrument to automatically perform continuous
    # measurements
    my_instrument.write(':INIT:CONT')

    # * Switch the instrument triggering function to 'EXT'
    # in order to control the measurements remotely
    my_instrument.write(':TRIG:SOUR EXT')
    time.sleep(5)


    my_instrument.write(':CORR:OPEN:STAT ON')
    my_instrument.write(':CORR:OPEN:EXEC')
    
    return print('the correction for the Open Circuit Potential was successfully collected')

___
___


### Run these cells to connect the instrument and interface with in using the code 

In [9]:
def instrument_conenction():
    
    """
    """
    # Connect the instrument and ensure you can comunicate wiht it
    rm = pyvisa.ResourceManager()
    if len(rm.list_resources()) != 0:
        print('The is/are \033[1m{}\033[0m insturment(s) conencted to this device.'.format(len(rm.list_resources())))

        # The argument of the following line is hardcoded to match our instrument. 
        # my_instrument = rm.open_resource('USB0::0x0957::0x0909::MY46205278::INSTR')

        # If only one instrument is connected to the computer, the argument can be replaced
        my_instrument = rm.open_resource(rm.list_resources()[0])

        # The next line is only to confirm that the correct instrument is connected to the computer
        print('\nThe selected instrument is:\033[1m {}'.format(my_instrument.query('*IDN?')))


        # Define the termination format - new line
        my_instrument.read_termination = '\n'
        my_instrument.write_termination = '\n'
        my_instrument.query('*IDN?')
        my_instrument.timeout= 100000

        return my_instrument
    else:
        return print('No instrument was found.\nEnsure the instrument is correctly connected to your device and try again')
        

In [10]:
LCR = instrument_conenction()

The is/are 1 insturment(s) conencted to this device.

The selected instrument is:  Agilent Technologies,E4980A,MY46205278,A.02.20



### Fil out the cell below with the information of the specific run


#### Define all the run variables in the cell below

In [11]:
# Experimental Conditions
function= 'ZTR'
voltage= '0.01'
# current = '0.002'

# Frequency sweep
freq_list= freq_gen(2E+5, 20, pts_per_decade=10)
freq_str= get_freq_str(freq_list)

# File information to use as the filename
sample = 'stand-0.01M-freq' 
temp =  '20C'
dist = '1mm'
# the save location. 
save_location = './EIS_Rheometer/'
comment = 'testing comment section'

### Then run all these cells without any additional change

In [14]:
EIS_measurements = EIS_measurement(LCR, function, voltage)
date = time.strftime('%y%m%d', time.localtime())
filename = date + '_EIS_' + sample + '_' + dist + '_' + temp  

write_file(freq_list, EIS_measurements, filename, save_location, comment)

The file was correctly saved


In [20]:
mode= 'ON'


if mode == 'on' or 'On'or'ON':
    print('ok')

ok
